In [15]:
import requests                         # For api get requests.
import datetime                         # For converting UTX into readable DateTime.
import toml                             # For interpreting Config file(s).
import json                             # For interpreting mining pool data ("pools.json").
from IPython.display import Markdown    # For improving visual experience of cell outputs.
from collections import Counter         # For counting address frequencies within a block.

In [16]:
# Import the api configuration string data.
with open('blockchain_api_config.toml', 'r') as f:
    api_config = toml.loads(f.read())

In [17]:
# Import known mining pool addresses.
with open('pools.json', 'r') as f:
    mining_pools = json.load(f)

<h1>At the time of your writing, what is the height of the most recent block mined and what is the
difficulty level?</h1>

In [18]:
def get_difficulty(_hash : str) -> int:
    """
    Gets the difficulty of a blockchain block in terms of the 0-requirement for a successful hash.
    :param _hash: (str) The block hash.
    :return: (int) The difficulty of the block.
    """
    _difficulty : int = 0
    for _letter in _hash:
        if _letter == "0":
            _difficulty += 1
        else:
            return _difficulty

In [19]:
# Get the latest block
latest_block = requests.get(api_config['latest_block']['root']).json()
hash_difficulty = requests.get(api_config['blockchain_api']['root'] + "q/getdifficulty").json()

height = latest_block['height']
difficulty = get_difficulty(latest_block['hash'])

Markdown(f"The height of the block is: <b>{height}</b>  "
         f"\nThe difficulty-level is: <b>{round(hash_difficulty/1000000000000, 2)} trillion</b>.  "
         f"\nThis is represented by <b>{difficulty}</b> initial '0' digits required for the hash of the block to be considered a successful hash.  "
         f"\nThe hash of the latest block is: <b>{latest_block['hash']}</b>")

The height of the block is: 870267  
The difficulty-level is: 101.65 trillion.  
This is represented by 19 initial '0' digits required for the hash of the block to be considered a successful hash.

<h1>What is the address of the miner of the block ? Can you unmask the identity of the
address?</h1>

In [20]:
def get_miner_addr(_tx : dict) -> str:
    """
    Takes a single transaction and returns the address of the miner.
    :param _tx: (dict) Transaction to get the miner address from.
    :return: (string) The address of the miner.
    """
    for _out in _tx['out']:
        if _out['addr']:
            return str(_out['addr'])
        else:
            pass
        
    print("Error finding address for output.")

In [21]:
def unmask_pool_name(_addr : str) -> str:
    """
    Checks known addresses of mining pools and returns that name if it exists.
    :param _addr: (str) Address of mining pool. Eg: 15MdAHnkxt9TMC2Rj595hsg8Hnv693pPBB
    :return: (str) The name of the mining pool. Eg: Mara Pool.
    
    If pool name is not located, returns 'Unknown'
    """
    for _ in list(mining_pools['payout_addresses'].keys()):
        if _addr == _:
            return mining_pools['payout_addresses'][_]['name']
    else:
        return 'Unknown'

In [22]:
# Access our transaction data and locate the first transaction which is our coinbase transaction in BTC blockchain.
txs = latest_block['txIndexes']
tx = requests.get(api_config['single_transaction']['root'] + str(txs[0])).json()

miner_addr = None

# Slight error checking in the illogical case that a non-coinbase transaction is obtained instead.
if len(tx['inputs']) == 1:
    miner_addr = get_miner_addr(tx)
else:
    print("Error: Coinbase Transaction not the selected transaction")
    
# See if we can identify if any known mining pools mined the block.
pool_name = unmask_pool_name(miner_addr)

Markdown(f"The Miner's Address is: <b>{miner_addr}</b>  "
         f"\nThe pool name is <b>{pool_name}</b>.")

The Miner's Address is: 1PuJjnF476W3zXfVYmJfGnouzFDAXakkL4  
The pool name is Unknown.

In [23]:
# Proof that the unmasking function does work with a known address which should return Mara Pool.
# Here is proof it should return Mara Pool: https://www.blockchain.com/explorer/addresses/btc/15MdAHnkxt9TMC2Rj595hsg8Hnv693pPBB
r = unmask_pool_name("15MdAHnkxt9TMC2Rj595hsg8Hnv693pPBB")
print(r)
del r

MARA Pool


<h1>How many transactions does it contain?</h1>

In [21]:
# Since txs variable is a List type, we can just call the length of the list.
tx_num = len(txs)

Markdown(f"The number of transactions in the most recent block is: <b>{tx_num}</b>")

The number of transactions in the most recent block is: 2628

<h1>Which address has the most balance?</h1>

<i>Defining Functions</i>

In [102]:
def get_block_addresses(_current_block : dict, _in : bool) -> list:
    """
    Takes the current block and finds all addresses referenced within the block; addresses who either sent btc (_in=True) or received btc (_in=False).
    :param _current_block: (dict) The dictionary representing the current block.
    :param _in: (bool) True if you want to look at transaction inputs. False if you want to look at transaction outputs.
    :return: (list) Return a list of all the addresses of the given _in parameter.
    """
    _addresses = []
    _txs = _current_block['txIndexes']
    
    if _in:
        for _tx in _txs[1:]:
            _req = requests.get(api_config['single_transaction']['root'] + str(_tx)).json()
            
            for _input in _req['inputs']:
                _addresses.append(_input['prev_out']['addr'])
                
    else:
        for _tx in _txs:
            _req = requests.get(api_config['single_transaction']['root'] + str(_tx)).json()
            
            for _out in _req['out']:
                
                if _out['value'] == 0:
                    continue
                elif 'addr' in _out.keys():
                    _addresses.append(_out['addr'])
                else:
                    continue
                
    return _addresses        

In [103]:
def addresses_to_str(_addresses : list) -> str:
    """
    Takes a list of addresses and converts it to a single string containing said addresses separated by pipes.
    :param _addresses: (list) List of addresses to convert.
    :return: (str) The string of all concatenated addresses separated by pipes.
    """
    _rtrn_str : str = ""
    
    for _addr in _addresses:
        
        if _rtrn_str == "":
            _rtrn_str = str(_addr)
            
        else:
            _rtrn_str += "|" + str(_addr)
            
    return _rtrn_str

In [104]:
def chunk_addresses(_addresses : list, _chunk_size : int = 50) -> list:
    """
    Takes a list of addresses and splits it into chunks of _chunk_size.
    For example, if you pass a list of 1000 it will return blocks of addresses equal to _chunk_size, default 50.
    
    This function should only be used in a loop as it returns a generator that yields chunks of addresses.
    
    :param _addresses: (list) List of addresses to split.
    :param _chunk_size: (int) The value of addresses to be grouped and returned.
    :return: (generator) List of addresses of a number equal to _chunk_size. 
    """
    for i in range(0, len(_addresses), _chunk_size):
        yield _addresses[i:i + _chunk_size]

In [105]:
def get_max_balance(_current_block : dict, _in : bool, _chunk_size : int = 50) -> dict:
    """
    Takes the current block and finds the maximum balance of the addresses in the block.
    :param _current_block: (dict) The dictionary representing the current block.
    :param _in: (bool) True if you want to look at transaction inputs. False if you want to look at transaction outputs.
    :param _chunk_size: (int) The value of addresses to be grouped and returned. Higher means more latency time but less compute time.
    :return: (dict) The address which held the maximum balance as the key, and the maximum balance as the value.
    """
    
    _addresses = get_block_addresses(_current_block, _in)
    
    # We get a lot of double-counts through the get_block_addresses
    # Since we aren't worried about transaction order, we can remove them to minimise api get requests.
    _unique_addr = list(set(_addresses))
    
    # Initialise our important counter/container variables.
    rtrn_dict : dict = {}
    max_balance : float = 0.0
    max_addr : str = ""
    
    # Iterate over our generator object returned from chunk_addresses() function.
    for _addr_chunk in chunk_addresses(_unique_addr, _chunk_size):
        # Turn our addresses into concatenated string which can be used in an api get request.
        _address_str = addresses_to_str(_addr_chunk)
        
        # Get the balance request of multiple addresses using pipe delimiter.
        balance_request = requests.get(api_config['balance']['root'] + _address_str)
        balance_data = balance_request.json()
        
        # For each address, look at the final balance of the address and see if it is the greatest value seen.
        for _addr, _bal_info in balance_data.items():
            balance = _bal_info['final_balance']
            max_balance = max(max_balance, balance)
            
            # If the max_balance is the balance for the current address, store that address.
            if balance == max_balance:
                max_addr = _addr
    
    
    rtrn_dict[max_addr] = max_balance        
            
    return rtrn_dict

In [106]:
def get_exchange_rates() -> dict:
    # Very simply gets exchangerates from the blockchain.
    return requests.get("https://blockchain.info/ticker").json()

In [107]:
def calc_currency_conversion(_exchange_data : dict, _currency : list, _btc : float) -> float:
    """
    Takes the exchange rate data and calculates the currency conversion.
    :param _exchange_data: (dict) The exchange rate data.
    :param _currency: (list) The currency to convert and crucially the time period for the data.
    :param _btc: (float) The btc value to convert. Crucially, not in satoshi.
    :return: (float) The currency conversion.
    """
    return _btc * int(_exchange_data[_currency[0]][_currency[1]])

In [108]:
def format_float_to_str(_float : float) -> str:
    # Converts a float value to string using string formatting.
    return "{:,.2f}".format(_float)

<i>End of defining functions</i>

In [109]:
exchange_data = get_exchange_rates()

In [111]:
in_max = get_max_balance(latest_block, True, 50)

In [113]:
in_max_addr, in_max_balance = next(iter(in_max.items()))

In [128]:
in_max_btc = in_max_balance / 100000000
in_max_usd = calc_currency_conversion(exchange_data, ["USD", "15m"], in_max_btc)
in_max_usd_f = format_float_to_str(in_max_usd)


Markdown(f"Max btc balance of addresses who sent money in this block is: <b>{in_max_btc}</b>  "
         f"\nThe USD conversion equates to: <b>${in_max_usd_f}</b>  "
         f"\nThe address with this value of bitcoin was: <b>{in_max_addr}</b>")

Max btc balance of addresses who sent money in this block is: 18208.77531143  
The USD conversion equates to: $1,546,616,957.40  
The address with this value of bitcoin was: bc1qm34lsc65zpw79lxes69zkqmk6ee3ewf0j77s3h

In [116]:
out_max = get_max_balance(latest_block, False, 50)

In [117]:
out_max_addr, out_max_balance = next(iter(out_max.items()))

In [129]:
out_max_btc = out_max_balance / 100000000
out_max_usd = calc_currency_conversion(exchange_data, ["USD", "15m"], out_max_btc)
out_max_usd_f = format_float_to_str(out_max_usd)


Markdown(f"Max btc balance of addresses who received money in this block is: <b>{out_max_btc}</b>  "
         f"\nThe USD conversion equates to: <b>${out_max_usd_f}</b>  "
         f"\nThe address with this value of bitcoin was: <b>{out_max_addr}</b>")

Max btc balance of addresses who received money in this block is: 27490.96467139  
The USD conversion equates to: $2,335,027,557.26  
The address with this value of bitcoin was: bc1qr4dl5wa7kl8yu792dceg9z5knl2gkn220lk7a9

In [134]:
if out_max_btc > in_max_btc:
    print(f"Therefor, the address with the greatest balance is {out_max_addr} with {out_max_btc} BTC")
    
else:
    print(f"Therefor, the address with the greatest balance is {in_max_addr} with {in_max_btc} BTC")

<h1>Which address has most number of transactions?</h1>

In [119]:
total_addr = get_block_addresses(latest_block, True)
total_addr += get_block_addresses(latest_block, False)

In [137]:
addr_count = Counter(total_addr)

max_addr, max_count = addr_count.most_common(1)[0]

Markdown(f"The address with the most transactions in this block was: {max_addr}  "
         f"\nThey were involved in transactions (potentially as both senders and receivers) a total of {max_count} times in this block!")

The address with the most transactions in this block was: 1GrwDkr33gT6LuumniYjKEGjTLhsL5kmqC  
They were involved in transactions (potentially as both senders and receivers) a total of 556 times in this block!

<h1>When did this address become active? (first transaction on the network)</h1>

In [147]:
seen = requests.get(api_config['blockchain_api']['root'] + 'q/addressfirstseen/' + str(max_addr)).json()
seen_dt = datetime.datetime.utcfromtimestamp(seen)

Markdown(f"This address saw its first activity on the Bitcoin blockchain on "
         f"{seen_dt.strftime('%d/%m/%y')} at {seen_dt.strftime('%H:%M')}.")

This address saw its first activity on the Bitcoin blockchain on 06/08/21 at 06:32.